# CPDCTL Samples for Notebooks and Environments in Projects

CPDCTL is a command-line interface (CLI) you can use to manage the lifecycle of notebooks. By using the notebook CLI, you can automate the flow for creating notebooks and running notebook jobs, moving notebooks between projects in Watson Studio, and adding custom libraries to notebook runtime environments.   

This notebook begins by showing you how to install and configure CPDCTL and is then split up into four sections with examples of how to use the commands for:

- Creating notebooks and running notebook jobs
- Creating Python scripts and running script jobs
- Downloading notebooks from one project and uploading them to another project
- Adding custom libraries to a notebook runtime environment

## Table of Contents

[1. Installing and configuring CPDCTL](#part1)
- [1.1 Installing the latest version of CPDCTL](#part1.1)
- [1.2 Adding CPD cluster configuration settings](#part1.2)

[2. Demo 1: Creating a notebook asset and running a job](#part2)
- [2.1 Creating a notebook asset](#part2.1)
- [2.2 Running a job](#part2.2)

[3. Demo 2: Creating a Python script asset and running a job](#part3)
- [3.1 Creating a Python script asset](#part3.1)
- [3.2 Running a job](#part3.2)

[4. Demo 3: Downloading a notebook and uploading it to another project](#part4)
- [4.1 Downloading a notebook](#part4.1)
- [4.2 Uploading the notebook to another project](#part4.2)

[5. Demo 4: Adding additional packages to custom environment](#part5)
- [5.1 Creating a custom software specification](#part5.1)
- [5.2 Adding additional packages](#part5.2)
- [5.3 Creating a custom environment](#part5.3)

## Before you begin
Import the following libraries:

In [1]:
import base64
import json
import os
import requests
import platform
import tarfile
import zipfile
from IPython.core.display import display, HTML

##  1. Installing and configuring CPDCTL <a class="anchor" id="part1"></a>

### 1.1 Installing the latest version of CPDCTL <a class="anchor" id="part1.1"></a>

To use the notebook and environment CLI commands, you need to install CPDCTL. Download the binary from the [CPDCTL GitHub respository](https://github.com/IBM/cpdctl/releases).

Download the binary and then display the version number:

In [2]:
PLATFORM = platform.system().lower()
CPDCTL_ARCH = "{}_amd64".format(PLATFORM)
CPDCTL_RELEASES_URL="https://api.github.com/repos/IBM/cpdctl/releases"
CWD = os.getcwd()
PATH = os.environ['PATH']
CPD_CONFIG = os.path.join(CWD, '.cpdctl.config.yml')

response = requests.get(CPDCTL_RELEASES_URL)
assets = response.json()[0]['assets']
platform_asset = next(a for a in assets if CPDCTL_ARCH in a['name'])
cpdctl_url = platform_asset['url']
cpdctl_file_name = platform_asset['name']

response = requests.get(cpdctl_url, headers={'Accept': 'application/octet-stream'})
with open(cpdctl_file_name, 'wb') as f:
    f.write(response.content)
    
display(HTML('<code>cpdctl</code> binary downloaded from: <a href="{}">{}</a>'.format(platform_asset['browser_download_url'], platform_asset['name'])))

In [3]:
%%capture

%env PATH={CWD}:{PATH}
%env CPD_CONFIG={CPD_CONFIG}

In [4]:
if cpdctl_file_name.endswith('tar.gz'):
    with tarfile.open(cpdctl_file_name, "r:gz") as tar:
        tar.extractall()
elif cpdctl_file_name.endswith('zip'):
    with zipfile.ZipFile(cpdctl_file_name, 'r') as zf:
        zf.extractall()

if CPD_CONFIG and os.path.exists(CPD_CONFIG):
    os.remove(CPD_CONFIG)
    
version_r = ! cpdctl version
CPDCTL_VERSION = version_r.s

print("cpdctl version: {}".format(CPDCTL_VERSION))

cpdctl version: 1.1.132


### 1.2  Adding CPD cluster configuration settings <a class="anchor" id="part1.2"></a>

Before you can use CPDCTL, you need to add configuration settings. You only need to configure these settings once for the same IBM Cloud Pak for Data (CPD) user and cluster. Begin by entering your CPD credentials and the URL to the CPD cluster:

In [ ]:
CPD_USER_NAME = 'dhshi'
CPD_USER_PASSWORD = 'passw0rd'
CPD_URL = 'https://cpd-cpd-instance.apps.cp4d404ugi.cp.fyre.ibm.com'

Add "cpd_user" user to the cpdctl configuration:

In [7]:
! cpdctl config user set cpd_user --username {CPD_USER_NAME} --password {CPD_USER_PASSWORD}

Add "cpd" cluster to the cpdctl configuration:

In [8]:
! cpdctl config profile set cpd --url {CPD_URL}

Add "cpd" context to the cpdctl configuration:

In [9]:
! cpdctl config context set cpd --profile cpd --user cpd_user

List available contexts:

In [10]:
! cpdctl config context list

Name                          Profile                       User                       Current   
inClusterEnvironmentContext   inClusterEnvironmentProfile   inClusterEnvironmentUser   *   


Switch to the context you just created if it is not marked in the `Current` column:

In [12]:
! cpdctl config context use inClusterEnvironmentContext

Switched to context "inClusterEnvironmentContext".


List available projects in context:

In [29]:
! cpdctl project list

...
ID                                     Name                                                Created                    Description                                          Tags   
0619c2d3-2b75-42f7-97c1-9d898fdef44c   Mortgage default project                            2022-02-11T12:02:58.633Z                                                        []   
19a29ada-3e9c-4147-bf35-4cf431ed3a26   AutoAI-TD-Sub                                       2022-01-25T01:20:57.078Z   The classification goal to train a model that can…   []   
25fc9237-bda9-42f9-ab83-124bb92b3ae9   cpdctl-samples-project-notebook-project-test        2022-04-03T15:46:00.738Z                                                        []   
2ba058a6-4ef7-4b3b-acde-44baaf288efb   test-analytics-project-git-ibm-sk-1                 2022-04-01T13:10:46.824Z                                                        []   
32d75b88-b113-4bd4-9502-cd2f471a8623   MLOps-CPD3.0-SWAT-StarterKit-AutoAI-Project-v0.96   2022-01-24T16:02:0

Choose the project in which you want to work:

In [30]:
result = ! cpdctl project list --output json -j "(resources[].metadata.guid)[0]" --raw-output
# project_id = result.s
print("project id: {}".format(project_id))

# You can also specify your project id directly:
project_id = "25fc9237-bda9-42f9-ab83-124bb92b3ae9"
print("project id: {}".format(project_id))

project id: 635a5511-72f6-4eb3-8ec6-0763a25e39cb
project id: 25fc9237-bda9-42f9-ab83-124bb92b3ae9


## 2. Demo 1: Creating a notebook asset and running a job <a class="anchor" id="part2"></a>

Before starting with this section, ensure that you have run the cells in [Section 1](#part1) and specified the ID of the project in which you will work.

Suppose you have a Jupyter Notebook (.ipynb) file on your local system and you would like to run the code in the file as a job on a CPD cluster. This section shows you how to create a notebook asset and run a job on a CPD cluster. 

### 2.1 Creating a notebook asset<a class="anchor" id="part2.1"></a>

First of all, you need to create a notebook asset in your project. To create a notebook asset you need to specify:
- The environment in which your notebook is to run
- A notebook file (.ipynb).

List all the environments in your project, filter them by their display name and get the ID of the environment in which your notebook will be run:

In [31]:
environment_name = "Default Python 3.8"
query_string = "(resources[?entity.environment.display_name == '{}'].metadata.asset_id)[0]".format(environment_name)

In [32]:
result = ! cpdctl environment list --project-id {project_id} --output json -j "{query_string}" --raw-output
env_id = result.s
print("environment id: {}".format(env_id))

# You can also specify your environment id directly:
# env_id = "Your environment ID"

environment id: jupconda38-25fc9237-bda9-42f9-ab83-124bb92b3ae9


Upload the .ipynb file:

In [39]:
remote_file_path = "notebook/cpdctl-test-notebook.ipynb"
local_file_path = "Connections-samples.ipynb"

In [40]:
! cpdctl asset file upload --path {remote_file_path} --file {local_file_path} --project-id {project_id}

...
OK


Create a notebook asset:

In [41]:
file_name = "cpdctl-test-notebook.ipynb"
runtime = {
    'environment': env_id
}
runtime_json = json.dumps(runtime)
originate = {
    'type': 'blank'
}
originate_json = json.dumps(originate)

In [42]:
result = ! cpdctl notebook create --file-reference {remote_file_path} --name {file_name} --project {project_id} --runtime '{runtime_json}' --originates-from '{originate_json}' --output json -j "metadata.asset_id" --raw-output
notebook_id = result.s
print("notebook id: {}".format(notebook_id))

notebook id: 2b8ccc18-55c4-47dd-b33e-d6b66a760fa8


### 2.2 Running a job <a class="anchor" id="part2.2"></a>

Before creating a notebook job, you need to create a version of your notebook:

In [43]:
result = ! cpdctl notebook version create --notebook-id {notebook_id} --output json -j "metadata.guid" --raw-output
version_id = result.s
print("version id: {}".format(version_id))

version id: 9bd58f65-80c3-4eb8-b82c-5a49f6f3ef8a


To create a notebook job, you need to give your job a name, add a description, and pass the notebook ID and environment ID you determined in [2.1](#part2.1). Additionally, you can add environment variables that will be used in your notebook:

In [44]:
job_name = "cpdctl-test-job"
job = {
    'asset_ref': notebook_id, 
    'configuration': {
        'env_id': env_id, 
        'env_variables': [
            'foo=1', 
            'bar=2'
        ]
    }, 
    'description': 'my job', 
    'name': job_name
}
job_json = json.dumps(job)

In [45]:
result = ! cpdctl job create --job '{job_json}' --project-id {project_id} --output json -j "metadata.asset_id" --raw-output
job_id = result.s
print("job id: {}".format(job_id))

job id: 1c0bb5a2-cbb2-4619-91a3-0b8c85b39914


Run a notebook job:

In [46]:
job_run = {
    'configuration': {
        'env_variables': [
            'key1=value1', 
            'key2=value2'
        ]
    }
}
job_run_json = json.dumps(job_run)

In [47]:
result = ! cpdctl job run create --project-id {project_id} --job-id {job_id} --job-run '{job_run_json}' --output json -j "metadata.asset_id" --raw-output
run_id = result.s
print("run id: {}".format(run_id))

run id: 36971249-e630-4c4f-8050-aab304aa397a


You can see the output of each cell in your .ipynb file by listing job run logs:

In [48]:
! cpdctl job run logs --job-id {job_id} --run-id {run_id} --project-id {project_id}

...

Cell 6:

Cell 9:
cpdctl version: 1.1.132

Cell 19:
Name                          Profile                       User                       Current   
inClusterEnvironmentContext   inClusterEnvironmentProfile   inClusterEnvironmentUser   *   

Cell 21:
Switched to context "inClusterEnvironmentContext".

Cell 23:
...

ID                                     Name                                                Created                    Description                                          Tags   
0619c2d3-2b75-42f7-97c1-9d898fdef44c   Mortgage default project                            2022-02-11T12:02:58.633Z                                                        []   
19a29ada-3e9c-4147-bf35-4cf431ed3a26   AutoAI-TD-Sub                                       2022-01-25T01:20:57.078Z   The classification goal to train a model that can…   []   
25fc9237-bda9-42f9-ab83-124bb92b3ae9   cpdctl-samples-project-notebook-project-test        2022-04-03T15:46:00.738Z                              

## 3. Demo 2: Creating a Python script asset and running a job <a class="anchor" id="part3"></a>

Before starting with this section, ensure that you have run the cells in [Section 1](#part1) and specified the ID of the project in which you will work.

Suppose you have a Python script (.py) on your local system and you would like to run the code in the script as a job on a CPD cluster. This section shows you how to create a Python script asset and run a job on a CPD cluster.

### 3.1 Creating a Python script asset<a class="anchor" id="part3.1"></a>

Upload the script (.py) file:

In [51]:
remote_file_path = "script/test_script.py"
local_file_path = "test_script.py"

In [52]:
! cpdctl asset file upload --path {remote_file_path} --file {local_file_path} --project-id {project_id}

...
OK


Specify the metadata, entity and attachments of the script file:

In [53]:
metadata = {
    "name": "my_test_script",
    "asset_type": "script",
    "asset_category": "USER",
    "origin_country": "us"
}
metadata_json = json.dumps(metadata)

entity = {
    "script": {
        "language": {
            "name": "python3"
        }
    }
}
entity_json = json.dumps(entity)

attachments = [
    {
        "asset_type": "script",
        "name": "my_test_script",
        "description": "attachment for script",
        "mime": "application/text",
        "object_key": remote_file_path
    }
]
attachments_json = json.dumps(attachments)

Create a Python script asset:

In [54]:
result = ! cpdctl asset create  --metadata '{metadata_json}' --entity '{entity_json}' --attachments '{attachments_json}' --project-id {project_id} --output json -j "metadata.asset_id" --raw-output
script_id = result.s
print("script id: {}".format(script_id))

script id: 991b7c56-5d07-4ef6-8681-e4d86e462a3b


### 3.2 Running a job<a class="anchor" id="part3.2"></a>

Similar to a notebook job, you need to specify the environment in which your script job is to run:

In [55]:
environment_name = "Default Python 3.8"
query_string = "(resources[?entity.environment.display_name == '{}'].metadata.asset_id)[0]".format(environment_name)

In [56]:
result = ! cpdctl environment list --project-id {project_id} --output json -j "{query_string}" --raw-output
env_id = result.s
print("environment id: {}".format(env_id))

# You can also specify your environment id directly:
# env_id = "Your environment ID"

environment id: jupconda38-25fc9237-bda9-42f9-ab83-124bb92b3ae9


Now you can create a script job. To do this, you need to give your script job a name, a description, and pass the script ID and environment ID.

In [57]:
job_name = "cpdctl-test-job-for-script"
job = {
    'asset_ref': script_id, 
    'configuration': {
        'env_id': env_id, 
        'env_variables': [
            'foo=1', 
            'bar=2'
        ]
    }, 
    'description': 'my script job', 
    'name': job_name
}
job_json = json.dumps(job)

In [58]:
result = ! cpdctl job create --job '{job_json}' --project-id {project_id} --output json -j "metadata.asset_id" --raw-output
job_id = result.s
print("job id: {}".format(job_id))

job id: 204f994d-512a-4af1-bcc1-97f636340648


Run your script job:

In [59]:
job_run = {
    'configuration': {
        'env_variables': [
            'key1=value1', 
            'key2=value2'
        ]
    }
}
job_run_json = json.dumps(job_run)

In [60]:
result = ! cpdctl job run create --project-id {project_id} --job-id {job_id} --job-run '{job_run_json}' --output json -j "metadata.asset_id" --raw-output
run_id = result.s
print("run id: {}".format(run_id))

run id: b8569a50-de46-4ef7-936d-dba0c72d6511


Show your script job run logs:

In [61]:
! cpdctl job run logs --job-id {job_id} --run-id {run_id} --project-id {project_id}

...

HelloCP4D 4.0.6




## 4. Demo 3: Downloading a notebook and uploading it to another project <a class="anchor" id="part4"></a>

Before starting with this section, ensure that you have run the cells in [Section 1](#part1) and specified the ID of the project in which you will work.

Suppose you have a notebook in one project and would like to add a specific version of this notebook to another project. To do this, you first need to download the notebook file to your local system and then upload it to the other project. After that you need to create a notebook asset in your project by referencing the uploaded notebook file (.ipynb) and specifying the environment in which your notebook is to run.

### 4.1 Downloading a notebook <a class="anchor" id="part4.1"></a>

You can select which notebook version you want to download.

List notebook versions:

In [62]:
! cpdctl notebook version list --notebook-id {notebook_id}

...
ID                                     Created   
9bd58f65-80c3-4eb8-b82c-5a49f6f3ef8a   1649002300317   


Get the path in the storage volume to the notebook version that you want to download:

In [63]:
result = ! cpdctl notebook version list --notebook-id {notebook_id} --output json -j "(resources[].metadata.guid)[0]" --raw-output
version_id = result.s
print("version id: {}".format(version_id))

# You can also specify your version id directly:
# env_id = "Your version ID"

version id: 9bd58f65-80c3-4eb8-b82c-5a49f6f3ef8a


In [64]:
result = ! cpdctl notebook version get --notebook-id {notebook_id} --version-id {version_id} --output json -j "entity.file_reference" --raw-output
version_storage_path = result.s
print("version storage path: {}".format(version_storage_path))

version storage path: notebook/attachment_for_notebook_2b8ccc18_55c4_47dd_b33e_d6b66a760fa8_7cibhro27aps0wqp1z4va9eze.ipynb


Download the noteboook asset with the specific version from the storage path:

In [65]:
file_name = "my-new-notebook.ipynb"

! cpdctl asset file download --path {version_storage_path} --output-file {file_name} --project-id {project_id} --raw-output

...
OK
Output written to my-new-notebook.ipynb


### 4.2 Uploading the notebook to another project <a class="anchor" id="part4.2"></a>

Determine the ID of the project to which you want to upload your notebook:

In [66]:
result = ! cpdctl project list --output json -j "(resources[].metadata.guid)[1]" --raw-output
project2_id = result.s
print("another project id: {}".format(project2_id))

# You can also specify your another project id directly:
# project2_id = "Your another project ID"

another project id: 19a29ada-3e9c-4147-bf35-4cf431ed3a26


Upload the notebook file to this project: my project: AutoAI-TD-Sub

In [67]:
remote_file_path = "notebook/{}".format(file_name)

! cpdctl asset file upload --path {remote_file_path} --file {file_name} --project-id {project2_id}

...
OK


After you have uploaded the notebook file to the project, you need to specify the environment in which to run the notebook:

In [68]:
environment_name = "Default Python 3.8"
query_string = "(resources[?entity.environment.display_name == '{}'].metadata.asset_id)[0]".format(environment_name)

In [69]:
result = ! cpdctl environment list --project-id {project2_id} --output json -j "{query_string}" --raw-output
env_id = result.s
print("environment id: {}".format(env_id))

# You can also specify your environment id directly:
# env_id = "Your environment ID"

environment id: jupconda38-19a29ada-3e9c-4147-bf35-4cf431ed3a26


Now you can create a notebook asset in this project by referencing the uploaded notebook file:

In [70]:
file_name = "my-new-notebook-in-another-project.ipynb"
runtime = {
    'environment': env_id
}
runtime_json = json.dumps(runtime)
originate = {
    'type': 'blank'
}
originate_json = json.dumps(originate)

In [71]:
result = ! cpdctl notebook create --file-reference {remote_file_path} --name {file_name} --project {project2_id} --originates-from '{originate_json}' --runtime '{runtime_json}' --output json -j "metadata.asset_id" --raw-output
notebook_id = result.s
print("notebook id: {}".format(notebook_id))

notebook id: 1b14155a-6705-40a1-ab6c-622be556e584


## 5. Demo 4: Adding additional packages for custom environment <a class="anchor" id="part5"></a>

Before starting with this section, ensure that you have run the cells in [Section 1](#part1) and specified the ID of the project in which you will work.

Suppose you have a `conda-yml` file that lists your additional packages **or** you have a `pip-zip` file containing your custom packages, and you would like to install these packages in your custom environment. To do this, you need to:

- Create a custom software specification
- Add your custom packages
- Create a custom environment

### 5.1 Creating a custom software specification <a class="anchor" id="part5.1"></a>

To create a custom software specification, you need to specify the base software specification that you want to customize. You can list all the software specifications in your project and choose one of them as the base software specification:

In [72]:
! cpdctl environment software-specification list --project-id {project_id} 

...
ID                                     Name                            Created                    Description                                          Type   
0062b8c9-8b7d-44a0-a9b9-46c416adcbd9   default_py3.6                   2022-01-17T22:11:48.813Z   Default Python 3.6                                   software_specification   
069ea134-3346-5748-b513-49120e15d288   pytorch-onnx_1.3-py3.7-edt      2022-01-17T22:11:48.823Z   Software specification for Pytorch 1.3.1 Elastic …   software_specification   
09c5a1d0-9c1e-4473-a344-eb7b665ff687   scikit-learn_0.20-py3.6         2022-01-17T22:11:48.815Z   Software specification for Scikit-learn on Python…   software_specification   
09f4cff0-90a7-5899-b9ed-1ef348aebdee   spark-mllib_3.0-scala_2.12      2022-01-17T22:11:48.853Z   Machine Learning on Spark 3.0 with Scala 2.12        software_specification   
0b848dd4-e681-5599-be41-b5f6fccc6471   pytorch-onnx_rt22.1-py3.9       2022-02-25T22:15:17.381Z   Software specification for Pyto

In [73]:
base_sw_spec_name = "Default Python 3.8"
query_string = "(resources[?metadata.description == '{}'].metadata.asset_id)[0]".format(base_sw_spec_name)

result = ! cpdctl environment software-specification list --project-id {project_id} --output json -j "{query_string}" --raw-output
base_sw_spec_id = result.s
print("base software specification id: {}".format(base_sw_spec_id))

# You can also specify your base software specification id directly:
# based_sw_spec_id = "Your base software specification ID"

base software specification id: ab9e1b80-f2ce-592c-a7d2-4f2344f77194


Create a custom software specification:

In [74]:
custom_sw_spec_name = "my_sw_spec"

base_sw_spec = {
    'guid': base_sw_spec_id
}

base_sw_spec_json = json.dumps(base_sw_spec)

sw_conf = {}
sw_conf_json = json.dumps(sw_conf)

In [75]:
result = ! cpdctl environment software-specification create --project-id {project_id} --name {custom_sw_spec_name} --base-software-specification '{base_sw_spec_json}' --software-configuration '{sw_conf_json}' --output json -j "metadata.asset_id" --raw-output
custom_sw_spec_id = result.s
print("custom software specification id: {}".format(custom_sw_spec_id))

custom software specification id: 6351dab8-e988-49e2-b9a6-84abae4de332


### 5.2 Adding additional packages <a class="anchor" id="part5.2"></a>

Create a package extension:

In [76]:
pkg_name = "my_test_packages"

In [77]:
result = ! cpdctl environment package-extension create --name {pkg_name} --type "conda_yml" --project-id {project_id}  --output json
pkg_ext_id = json.loads(result.s)['metadata']['asset_id']
print("package extension id: {}".format(pkg_ext_id))

package extension id: 232ed7dc-17ce-4897-bead-2396fdf86123


Get the path to where you want to upload the additional packages:

In [78]:
pkg_ext_href = json.loads(result.s)['entity']['package_extension']['href'].split('/')[4].split('?')[0]
remote_pkg_path = "package_extension/{}".format(pkg_ext_href)
print("path where asset should be uploaded: {}".format(remote_pkg_path))

path where asset should be uploaded: package_extension/my_test_packages_b4Bzd2Tvk.yml


Define a conda-yaml file listing additional packages:

In [79]:
my_yaml = """
channels:
  - defaults

dependencies:
  - pip:
    - fuzzywuzzy

"""

with open('my-pkg-ext.yaml', 'w') as f:
    f.write(my_yaml)

Upload additional packages to the path returned in the previous command:

In [80]:
local_pkg_path = "./my-pkg-ext.yaml"

In [81]:
! cpdctl asset file upload --path "{remote_pkg_path}" --file {local_pkg_path} --project-id {project_id} 

...
OK


In [82]:
! cpdctl environment package-extension upload-complete --package-extension-id {pkg_ext_id} --project-id {project_id}

...
OK


Add the package extension into the custom software specification:

In [83]:
! cpdctl environment software-specification add-package-extensions --software-specification-id {custom_sw_spec_id} --package-extension-id {pkg_ext_id} --project-id {project_id}

...
OK


### 5.3 Creating a custom environment <a class="anchor" id="part5.3"></a>

List all the hardware specifications in your project and choose one that fits your custom environment:

In [84]:
! cpdctl environment hardware-specification list --project-id {project_id} 

...
ID                                     Name            Created                    Description                                          Type   
5a1f0e64-e420-55ce-bd7a-f6d00bb942cf   ML              2022-01-17T22:11:49.321Z   A hardware specification providing 4 CPU cores an…   hardware_specification   
a02f3ab5-6964-4f06-a870-c7cc69187895   V100x2          2022-01-17T22:11:49.322Z   A hardware specification providing 52 CPU cores a…   hardware_specification   
a6c4923b-b8e4-444c-9f43-8a7ec3020110   L               2022-01-17T22:11:49.320Z   A hardware specification providing 8 CPU cores an…   hardware_specification   
ac59d20a-9c7c-4504-a853-788ef35969da   Default Spark   2022-01-17T22:11:49.319Z   A hardware specification for Spark with 1 CPU and…   hardware_specification   
b128f957-581d-46d0-95b6-8af5cd5be580   XXS             2022-01-17T22:11:49.324Z   A hardware specification providing one CPU core a…   hardware_specification   
b2232f7a-bfad-4822-9bce-6ba1af49217a   M-Spark  

In [85]:
hw_spec_keyword_1 = "one CPU core"
hw_spec_keyword_2 = "4 GiB of memory"
query_string = "(resources[?contains(metadata.description, '{}') && contains(metadata.description, '{}')].metadata.asset_id)[0]".format(hw_spec_keyword_1, hw_spec_keyword_2)

result = ! cpdctl environment hardware-specification list --project-id {project_id}  --output json -j "{query_string}" --raw-output
hw_spec_id = result.s
print("hardware specification id: {}".format(hw_spec_id))

# You can also specify your hardware specification id directly:
# hw_spec_id = "Your base software specification ID"

hardware specification id: f3ebac7d-0a75-410c-8b48-a931428cc4c5


Create a custom environment by specifying the hardware specification, the custom software specification and the tool specification:

In [86]:
env_name = "my_custom_env"
hw_spec = {
    'guid': hw_spec_id
}
custom_sw_spec = {
    'guid': custom_sw_spec_id
}
custom_sw_spec_json = json.dumps(custom_sw_spec)
tool_spec = {
    'supported_kernels': [{
        'language': 'python', 
        'version': '3.7', 
        'display_name': 'Python 3.7'
    }]
}
hw_spec_json = json.dumps(hw_spec)
tool_spec_json = json.dumps(tool_spec)

In [87]:
result = ! cpdctl environment create --project-id {project_id} --type "notebook" --name {env_name} --display-name {env_name} --hardware-specification '{hw_spec_json}' --software-specification '{custom_sw_spec_json}' --tools-specification '{tool_spec_json}' --output json -j "metadata.asset_id" --raw-output
custom_env_id = result.s
print("custom environment id: {}".format(custom_env_id))

custom environment id: 43306410-dae7-40de-87a7-3abf806473c8


Now you can use this custom environment when you create a new notebook asset.

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.